# Missing values

Missing data is the absence of values in certain observations of a variable. Missing data is an unavoidable problem in most data sources and may have a significant impact on the conclusions that we derived from the data.

`Why is the data missing?`

The source of missing data can vary. These are just some examples:

* The value was forgotten, lost, or not stored properly.

* The value does not exist.

* The value can't be known or identified.

To give real-life examples, a person may choose not to complete all fields in a form if they are not mandatory. That would introduce missing data. Sometimes people do not want to disclose some information, for example, income, or they do not know the answers to the questions being asked.

Sometimes the value for a certain variable does not exist. For example, in the variable "total debt as percentage of total income" (very common in financial data), if the person has no income, then the total percentage of 0 does not exist, and therefore it will be a missing value.

It's important to understand why data is missing, in other words, the mechanism of missing data. We may process the missing information differently depending on this mechanism. Furthermore, identifying the source of missing data allows us to take steps to regulate that source and reduce the amount of missing data as data collection progresses.

### Missing data mechanisms

There are 3 mechanisms that lead to missing data. Two of them involve missing data randomly and the third one involves a systematic loss of data.

`Missing Completely at Random, MCAR:`
If the likelihood of a value being missing is the same for all observations, the variable is missing completely at random (MCAR). The data points that are missing are a random subset of the observations.

If data is MCAR, then disregarding observations with missing data would not bias the inferences made.

`Missing at Random, MAR:`
If the probability of an observation being missing depends on available information (i.e., other variables), then the observation is missing at random (MAR). There is a relationship between the likelihood of a value being missing and the observed data.

For example, if men are more likely to disclose their weight than women, weight is MAR. The weight information will be missing at random for the men and women who did not disclose their weight, but as men are more prone to disclose it, there will be more missing values for women than for men.

In this example, missing data points are no longer a random subset of the total observations.

If we decide to proceed with the variable with missing values (in this case weight), we might want to include the variable gender to control the bias in weight for the missing observations.

`Missing Not at Random, MNAR:`
If there is a mechanism or a reason why data is missing, then that data is missing not at random (MNAR). For example, if people failed to fill out a depression survey because of their depression, those data points would be missing not at random. The missing data occurs due to the depression.

Depending on the mechanism by which missing values occur, we may choose different missing data imputation methods.

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# To display all columns in the dataset.
pd.set_option('display.max_columns', None)

In [2]:
# Let's load the titanic dataset.
data = pd.read_csv('titanic.csv')

# Let's inspect the first 5 rows.
data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [3]:
# the isnull() method plus the sum() method:

data.isnull().sum()

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

There are 263 missing values for Age, 1014 for Cabin and 2 for Embarked.

In [4]:
# to obtain the fraction of missing values:

data.isnull().mean()

pclass       0.000000
survived     0.000000
name         0.000000
sex          0.000000
age          0.200917
sibsp        0.000000
parch        0.000000
ticket       0.000000
fare         0.000764
cabin        0.774637
embarked     0.001528
boat         0.628724
body         0.907563
home.dest    0.430863
dtype: float64

In the variables Age there is 20% of data missing.

There is 77 percent of data missing in the variable Cabin, in which the passenger was traveling.

There is 0.2 percent of data missing in the field Embarked (the port from which the passenger boarded the Titanic).

### Missing data Not At Random (MNAR)
The missing values of the variables `age` and `cabin`, were introduced systematically. For many of those who did not survive, their age or their cabin remains unknown. The people who survived could have been otherwise asked for that information.

If data is MNAR, we could expect a greater number of missing values for people who did not survive.

In [5]:
# Let's create a binary variable that indicates if the value of cabin is missing.

data['cabin_null'] = np.where(data['cabin'].isnull(), 1, 0)

In [6]:
# Let's evaluate the percentage of missing values in
# cabin for the people who survived vs the non-survivors.

# The variable Survived takes the value 1 if the passenger
# survived, or 0 otherwise.

# Group data by Survived vs Non-Survived
# and find the percentage of NaN for Cabin.

data.groupby(['survived'])['cabin_null'].mean()

survived
0    0.873918
1    0.614000
Name: cabin_null, dtype: float64

In [7]:
# Another way of doing the above, with less lines of code
data['cabin'].isnull().groupby(data['survived']).mean()

survived
0    0.873918
1    0.614000
Name: cabin, dtype: float64

The percentage of missing values is higher for those who did not survive (87% vs 60% for survivors). This finding could support our hypothesis that the data is missing because after people died, the information could not be retrieved.

**Note:** `to truly understand whether the data is missing not at random, we would need to get extremely familiar with the way data was collected. Analysing datasets, can only point us in the right direction or help us make assumptions.`

In [8]:
# Let's do the same for the variable age:

data['age_null'] = np.where(data['age'].isnull(), 1, 0)
data.groupby(['survived'])['age_null'].mean()

survived
0    0.234858
1    0.146000
Name: age_null, dtype: float64

In [9]:
data['age'].isnull().groupby(data['survived']).mean()

survived
0    0.234858
1    0.146000
Name: age, dtype: float64

So, more missing data points for the people who did not survive. The analysis therefore suggests that there was a systematic loss of data: people who did not survive had more missing information. Presumably, the method chosen to gather the information contributes to the generation of this missing data.

### Missing data Completely At Random (MCAR)



In [10]:
#  In the titanic dataset, there are also missing values for the variable Embarked.
data[data['embarked'].isnull()]


,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,cabin_null,age_null
168,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN,6,NaN,NaN,0,0
284,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN,6,NaN,"Cincinatti, OH",0,0


These 2 women were traveling together. Miss Icard was the maid of Mrs. Stone.

A priori, there does not seem to be an indication that the missing information in the variable "embarked" is dependent on any other variable, and the fact that these women survived means that they could have been asked for this information.

It is very likely the values were lost at the time of building the dataset.

If these values are MCAR, the likelihood of data missing for these two women is the same as the likelihood of data missing for any other person on the Titanic. Of course, this will be hard, if possible at all, to prove.

### Missing data at Random (MAR)


In [11]:
# Let's load the dataset with just the 2 variables.

data = pd.read_csv('loan.csv', usecols=['employment', 'time_employed'])

data.head()

,employment,time_employed
0,Teacher,<=5 years
1,Accountant,<=5 years
2,Statistician,<=5 years
3,Other,<=5 years
4,Bus driver,>5 years


In [12]:
# Let's check the percentage of missing data.

data.isnull().mean()

employment       0.0611
time_employed    0.0529
dtype: float64

Both variables have roughly the same percentage of missing observations.

In [13]:
# lLt's insptect the different employment types.

# Number of different employments.
print('Number of employments: {}'.format(
    len(data['employment'].unique())))

# Examples of employments.
data['employment'].unique()


Number of employments: 12


array(['Teacher', 'Accountant', 'Statistician', 'Other', 'Bus driver',
       'Secretary', 'Software developer', 'Nurse', 'Taxi driver', nan,
       'Civil Servant', 'Dentist'], dtype=object)

In [14]:
# Let's inspect the variable time employed.

data['time_employed'].unique()

array(['<=5 years', '>5 years', nan], dtype=object)

The customer can't enter a value for employment time if they are not employed. They could be students, retired, self-employed, or something else. Note how these 2 variables are related to each other.

In [15]:
# Let's calculate the proportion of missing data in time_employed variable when
# customers declared employment.

# Customers who declared employment
t = data[~data['employment'].isnull()]

# Percentage of missing data in time employed
t['time_employed'].isnull().mean()


0.0005325380764724678

In [16]:
# Let's do the same for those borrowers who did not 
# report employment.

# Customers who did not declare employment.
t = data[data['employment'].isnull()]

# Percentage of missing data in time employed.
t['time_employed'].isnull().mean()

0.8576104746317512

The number of borrowers who have reported occupation and have missing values in time_employed is minimal. Customers who did not report an occupation, on the other hand, mostly show missing values in the time_employed variable.

This further supports the hypothesis that the missing values in employment are related to the missing values in time_employed.

This is an example of MAR.

# Cardinality

The values of a categorical variable are selected from a group of categories, also called labels. For example, in the variable gender the categories are male and female, whereas in the variable city the labels could be London, Manchester, Brighton, and so on.

Categorical variables can contain different numbers of categories. The variable "gender" contains only 2 labels, but a variable like "city" or "postcode" can contain a huge number of labels.

The number of different labels is known as cardinality. A high number of labels within a variable is known as high cardinality.

### Is high cardinality a problem?

High cardinality poses the following challenges:

* Variables with too many labels tend to dominate those with only a few labels, particularly in decision tree-based algorithms.

* High cardinality may introduce noise.

* Some of the labels may only be present in the training data set and not in the test set, so machine learning algorithms may     over-fit to the training set.

* Some labels may appear only in the test set, leaving the machine learning algorithms unable to perform a calculation over the   new (unseen) observation.

**Algorithms based on decision trees can be biased towards variables with high cardinality.**

In [17]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# The machine learning models.
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# To evaluate the models.
from sklearn.metrics import roc_auc_score

# To separate data into train and test.
from sklearn.model_selection import train_test_split

In [18]:
data = pd.read_csv('titanic.csv')

data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


The categorical variables are Name, Sex, Ticket, Cabin and Embarked.

**Note:** `Ticket and Cabin contain both letters and numbers, so they could be treated as Mixed Variables`

In [19]:
# Inspect the cardinality: the number of different labels.

print('Number of categories in the variable Name: {}'.format(
    len(data.name.unique())))

print('Number of categories in the variable Gender: {}'.format(
    len(data.sex.unique())))

print('Number of categories in the variable Ticket: {}'.format(
    len(data.ticket.unique())))

print('Number of categories in the variable Cabin: {}'.format(
    len(data.cabin.unique())))

print('Number of categories in the variable Embarked: {}'.format(
    len(data.embarked.unique())))

print('Total number of passengers in the Titanic: {}'.format(len(data)))

Number of categories in the variable Name: 1307
Number of categories in the variable Gender: 2
Number of categories in the variable Ticket: 929
Number of categories in the variable Cabin: 182
Number of categories in the variable Embarked: 4
Total number of passengers in the Titanic: 1309


While the variable Sex contains only 2 categories and the variable Embarked 4 (low cardinality), the variables Ticket, Name, and Cabin, as expected, contain a huge number of different labels (high cardinality).

In [20]:
data.cabin.unique()

array(['B5', 'C22', 'E12', 'D7', 'A36', 'C101', nan, 'C62', 'B35', 'A23',
       'B58', 'D15', 'C6', 'D35', 'C148', 'C97', 'B49', 'C99', 'C52', 'T',
       'A31', 'C7', 'C103', 'D22', 'E33', 'A21', 'B10', 'B4', 'E40',
       'B38', 'E24', 'B51', 'B96', 'C46', 'E31', 'E8', 'B61', 'B77', 'A9',
       'C89', 'A14', 'E58', 'E49', 'E52', 'E45', 'B22', 'B26', 'C85',
       'E17', 'B71', 'B20', 'A34', 'C86', 'A16', 'A20', 'A18', 'C54',
       'C45', 'D20', 'A29', 'C95', 'E25', 'C111', 'C23', 'E36', 'D34',
       'D40', 'B39', 'B41', 'B102', 'C123', 'E63', 'C130', 'B86', 'C92',
       'A5', 'C51', 'B42', 'C91', 'C125', 'D10', 'B82', 'E50', 'D33',
       'C83', 'B94', 'D49', 'D45', 'B69', 'B11', 'E46', 'C39', 'B18',
       'D11', 'C93', 'B28', 'C49', 'B52', 'E60', 'C132', 'B37', 'D21',
       'D19', 'C124', 'D17', 'B101', 'D28', 'D6', 'D9', 'B80', 'C106',
       'B79', 'C47', 'D30', 'C90', 'E38', 'C78', 'C30', 'C118', 'D36',
       'D48', 'D47', 'C105', 'B36', 'B30', 'D43', 'B24', 'C2', 'C65',


From the previous cell we can see that there are 148 different cabins, therefore the variable is highly cardinal.

**Rationale**: `the first letter indicates the deck on which the cabin was located, indicating both social class status and proximity to the Titanic's surface. Both are known to improve the probability of survival.`

In [21]:
# Let's capture the first letter of cabin.

data['Cabin_reduced'] = data['cabin'].astype(str).str[0]

data[['cabin', 'Cabin_reduced']].head()
# So the cardinality of the variable is reduced

,cabin,Cabin_reduced
0,B5,B
1,C22,C
2,C22,C
3,C22,C
4,C22,C


In [22]:
print('Number of categories in the variable Cabin: {}'.format(
    len(data.cabin.unique())))

print('Number of categories in the variable Cabin reduced: {}'.format(
    len(data.Cabin_reduced.unique())))


Number of categories in the variable Cabin: 182
Number of categories in the variable Cabin reduced: 9
